# Follow-up

This tab provides an overview of the clinical trial status, ie. number of patients, median follow-up times, total data, missing data, etc.

In [ ]:
import os
import sys
sys.path.append("C:\\Users\\Fabien Boux\\Code\\ClinLib")

from functions.config import Config
config = Config()
config.read()

from clinlib.database import Database
database = Database(config.get_value('database', section='PATH'), idlength=3)
database.add_resource({'metadata': os.path.join(config.get_value('database', section='PATH'), config.get_value('metadata', section='PATH'))})

In [ ]:
import pandas as pd
import numpy as np

metadata = database.get_metadata(which='all')

group = metadata['Group']
group_labels = group.unique()

df = pd.DataFrame([], columns=list(group_labels) + ['Overall'])

df.loc['Number of patients'] = [(metadata['Group'] == g).sum() for g in group_labels] + [len(metadata['Group'])]
df.loc['Median follow-up (days)'] = [(metadata['End'][metadata['Group'] == g] - metadata['Start'][metadata['Group'] == g]).median().days for g in group_labels] + [(metadata['End'] - metadata['Start']).median().days]

df

The following plot is a *swimmer plot*. This graph allows a fast overview of all data, it combines time to event (patient's treatment, death or end of the follow-up) and imaging (via lesion evolutions if available).
Note that some variables used in this graph can be defined in the ```Configuration``` tab (the follow-up time, list of potential visits, ...).

In [ ]:
from functions.graph import swimmer_plot

if config.is_key('followup_time'):
    followup_time = config.get_value('followup_time')
else:
    followup_time = 12
if config.is_key('followup_visits'):
    followup_visits = config.get_value('followup_visits')
else:
    followup_visits = None

swimmer_plot(database, followup_time=followup_time, followup_visits=followup_visits);